This notebook is there to give evidence that the formulaes used in `benchmark.metrics.crps` accurately estimate the covariance between the estimations of dependent CRPS.

This is done by comparing the covariances obtained by repeatedly drawing from the forecast distribution and recomputing the CRPS, with the covariances obtained with the formulaes (also averaged over multiple drawing from the distribution).

In [9]:
import numpy as np

from benchmark.metrics.crps import crps, crps_covariance

In [2]:
# The time to run this notebook will scale with:
# O(NUM_TRIALS * NUM_SAMPLES ** 3)

NUM_TRIALS = 1000000
NUM_SAMPLES = 25

# Variance of a single CRPS estimate

### Uniform distribution

In [3]:
%%time
target = np.random.random()

crps_values = []
for _ in range(NUM_TRIALS):
    samples = np.random.random(NUM_SAMPLES)
    crps_values.append(crps(np.array(target), samples))
measured_variance = np.var(crps_values, ddof=1)

formula_variances = []
for _ in range(NUM_TRIALS):
    samples = np.random.random(NUM_SAMPLES)
    formula_variances.append(crps_covariance(
        Xa=samples,
        ya=target,
        Xb=samples,
        yb=target,
    ))
mean_formula_variance = np.mean(formula_variances)

print(f"Measured = {measured_variance}")
print(f"Formula  = {mean_formula_variance}")
print(f"Error    = {100 * np.abs(mean_formula_variance - measured_variance) / measured_variance:.3f}%")

Measured = 0.00027645365384839775
Formula  = 0.0002764035543528844
Error    = 0.018%
CPU times: user 2min 41s, sys: 36.5 ms, total: 2min 41s
Wall time: 2min 41s


### Normal distribution

In [4]:
%%time
target = np.random.random()

crps_values = []
for _ in range(NUM_TRIALS):
    samples = np.random.standard_normal(NUM_SAMPLES)
    crps_values.append(crps(np.array(target), samples))
measured_variance = np.var(crps_values, ddof=1)

formula_variances = []
for _ in range(NUM_TRIALS):
    samples = np.random.standard_normal(NUM_SAMPLES)
    formula_variances.append(crps_covariance(
        Xa=samples,
        ya=target,
        Xb=samples,
        yb=target,
    ))
mean_formula_variance = np.mean(formula_variances)

print(f"Measured = {measured_variance}")
print(f"Formula  = {mean_formula_variance}")
print(f"Error    = {100 * np.abs(mean_formula_variance - measured_variance) / measured_variance:.3f}%")

Measured = 0.01057775227306777
Formula  = 0.010576824081399408
Error    = 0.009%
CPU times: user 2min 47s, sys: 55.4 ms, total: 2min 47s
Wall time: 2min 47s


# Covariance of two independent CRPS estimates

### Uniform distribution

In [5]:
%%time
target = np.random.random(2)

crps_values = []
for _ in range(NUM_TRIALS):
    samples = np.random.random((NUM_SAMPLES, 2))
    crps_values.append(crps(np.array(target), samples))
measured_covariance = np.cov(crps_values, rowvar=False, ddof=1)[0,1]

formula_covariances = []
for _ in range(NUM_TRIALS):
    samples = np.random.random((NUM_SAMPLES, 2))
    formula_covariances.append(crps_covariance(
        Xa=samples[:,0],
        ya=target[0],
        Xb=samples[:,1],
        yb=target[1],
    ))
mean_formula_covariance = np.mean(formula_covariances)

print(f"Measured = {measured_covariance}")
print(f"Formula  = {mean_formula_covariance}")

Measured = -2.982969806225829e-07
Formula  = -2.614016066430593e-07
CPU times: user 2min 55s, sys: 136 ms, total: 2min 55s
Wall time: 2min 55s


### Normal distribution

In [6]:
%%time
target = np.random.random(2)

crps_values = []
for _ in range(NUM_TRIALS):
    samples = np.random.standard_normal((NUM_SAMPLES, 2))
    crps_values.append(crps(np.array(target), samples))
measured_covariance = np.cov(crps_values, rowvar=False, ddof=1)[0,1]

formula_covariances = []
for _ in range(NUM_TRIALS):
    samples = np.random.standard_normal((NUM_SAMPLES, 2))
    formula_covariances.append(crps_covariance(
        Xa=samples[:,0],
        ya=target[0],
        Xb=samples[:,1],
        yb=target[1],
    ))
mean_formula_covariance = np.mean(formula_covariances)

print(f"Measured = {measured_covariance}")
print(f"Formula  = {mean_formula_covariance}")

Measured = -4.986207612162525e-06
Formula  = 2.3010414186101426e-06
CPU times: user 2min 54s, sys: 132 ms, total: 2min 54s
Wall time: 2min 54s


# Covariance of two dependent CRPS estimates

### Uniform distribution

In [7]:
%%time
target = np.random.random(2)
transform = np.array([[1, -0.5], [0, 1]])

crps_values = []
for _ in range(NUM_TRIALS):
    samples = np.random.random((NUM_SAMPLES, 2)) @ transform
    crps_values.append(crps(np.array(target), samples))
measured_covariance01 = np.cov(crps_values, rowvar=False, ddof=1)[0,1]
measured_covariance10 = np.cov(crps_values, rowvar=False, ddof=1)[1,0]

formula_covariances01 = []
formula_covariances10 = []
for _ in range(NUM_TRIALS):
    samples = np.random.random((NUM_SAMPLES, 2)) @ transform
    formula_covariances01.append(crps_covariance(
        Xa=samples[:,0],
        ya=target[0],
        Xb=samples[:,1],
        yb=target[1],
    ))
    formula_covariances10.append(crps_covariance(
        Xa=samples[:,1],
        ya=target[1],
        Xb=samples[:,0],
        yb=target[0],
    ))
mean_formula_covariance01 = np.mean(formula_covariances01)
mean_formula_covariance10 = np.mean(formula_covariances10)

print(f"Measured = {measured_covariance01} {measured_covariance10}")
print(f"Formula  = {mean_formula_covariance01} {mean_formula_covariance10}")
error01 = 100 * np.abs(mean_formula_covariance01 - measured_covariance01) / np.abs(measured_covariance01)
error10 = 100 * np.abs(mean_formula_covariance10 - measured_covariance10) / np.abs(measured_covariance10)
print(f"Error    = {error01:.3f}% {error10:.3f}%")

Measured = 0.0003553556273485834 0.0003553556273485834
Formula  = 0.00035366180262113954 0.00035366180262113954
Error    = 0.477% 0.477%
CPU times: user 5min 33s, sys: 164 ms, total: 5min 33s
Wall time: 5min 33s


### Normal distribution

In [8]:
%%time
target = np.random.random(2)
transform = np.array([[1, 0.5], [0, 1]])

crps_values = []
for _ in range(NUM_TRIALS):
    samples = np.random.standard_normal((NUM_SAMPLES, 2)) @ transform
    crps_values.append(crps(np.array(target), samples))
measured_covariance01 = np.cov(crps_values, rowvar=False, ddof=1)[0,1]
measured_covariance10 = np.cov(crps_values, rowvar=False, ddof=1)[1,0]

formula_covariances01 = []
formula_covariances10 = []
for _ in range(NUM_TRIALS):
    samples = np.random.standard_normal((NUM_SAMPLES, 2)) @ transform
    formula_covariances01.append(crps_covariance(
        Xa=samples[:,0],
        ya=target[0],
        Xb=samples[:,1],
        yb=target[1],
    ))
    formula_covariances10.append(crps_covariance(
        Xa=samples[:,1],
        ya=target[1],
        Xb=samples[:,0],
        yb=target[0],
    ))
mean_formula_covariance01 = np.mean(formula_covariances01)
mean_formula_covariance10 = np.mean(formula_covariances10)

print(f"Measured = {measured_covariance01} {measured_covariance10}")
print(f"Formula  = {mean_formula_covariance01} {mean_formula_covariance10}")
error01 = 100 * np.abs(mean_formula_covariance01 - measured_covariance01) / np.abs(measured_covariance01)
error10 = 100 * np.abs(mean_formula_covariance10 - measured_covariance10) / np.abs(measured_covariance10)
print(f"Error    = {error01:.3f}% {error10:.3f}%")

Measured = 0.004972679182542276 0.004972679182542276
Formula  = 0.004959576361471908 0.004959576361471908
Error    = 0.263% 0.263%
CPU times: user 5min 33s, sys: 160 ms, total: 5min 33s
Wall time: 5min 33s
